In [51]:
#--------- QLAP ---------#
import pandas as pd
import os
# import snowflake.sqlalchemy as snowsa
import sqlalchemy as sa
from pathlib import Path
import csv
import sqlparse
from ast import literal_eval
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [7]:
engine = sa.create_engine(
    snowsa.URL(
        account="vestiairecollective.eu-west-1",
        user=os.environ['DBT_USER'],
        password=os.environ['DBT_PASSWORD'],
#         warehouse=os.environ['WAREHOUSE'],
        warehouse='SERVICE_WH_DEFAULT',
        database=os.environ['DATABASE']
    )
)

def get_data_from_snow(query):
    df = pd.read_sql(query, con=engine)
    return df

NameError: name 'snowsa' is not defined

In [18]:
fp = "C:/Users/phill/git/lots/qlap"
sql_file = os.path.join(fp, "fct_query_history_lin_reg.sql")
sql = Path(sql_file).read_text()
csv_file = os.path.join(fp, "fct_query_history_lin_reg.csv")
table_file = os.path.join(fp, "dim_table_storage.csv")

In [12]:
print(csv_file)

C:/Users/phill/git/lots/qlap\fct_query_history_lin_reg.csv


In [78]:
# initialize list elements
data = {
    'warehouse_size':['X-Small','Small','Medium','Large','X-Large'],
    'warehouse_multiplier':[1,2,4,8,16]
}

df_wh = pd.DataFrame.from_dict(data)
df_wh

,warehouse_size,warehouse_multiplier
0,X-Small,1
1,Small,2
2,Medium,4
3,Large,8
4,X-Large,16


In [79]:
if os.path.isfile(csv_file):
    df = pd.read_csv(csv_file, lineterminator='\n')
else:
    df = get_data_from_snow(sql)
#     df = df.replace(r'\n',' ', regex=True)
    df.query_text = df.query_text.replace(r'\n|"',' ', regex=True)
    df.tables_scan_list = df.tables_scan_list.replace(r'\n',' ', regex=True)
#     to_replace=r'^ba.$'
    df.to_csv(fp + '/fct_query_history_lin_reg.csv', quoting=csv.QUOTE_MINIMAL)

df.merge(df_wh)
    
df.head()

,Unnamed: 0,query_id,query_text,join_list,tables_scan_list,aggregate_list,query_type,query_load_percent,user_name,warehouse_size,...,gb_spilled_to_local_storage,gb_spilled_to_remote_storage,gb_written,gb_written_to_result,rows_produced,rows_inserted,rows_deleted,rows_updated,rows_unloaded,execution_status
0,0,01a815ab-3201-fbbf-0000-27695f99ba56,INSERT INTO DWH_OPS.#Tableau_101782_1_Connect_...,[],"[\n ""DWH_OPS.#Tableau_101782_1_Connect_CheckC...",[],INSERT,25.0,PAUL.PROVOST@VESTIAIRECOLLECTIVE.COM,Medium,...,0.0,0.0,9.536740e-07,0.0,1.0,1,0,0,0,SUCCESS
1,1,01a8165b-3201-fbc5-0000-27695f9ecb9a,INSERT INTO DWH_PRD.#Tableau_3_1_Connect_Check...,[],"[\n ""DWH_PRD.#Tableau_3_1_Connect_CheckCreate...",[],INSERT,50.0,PAUL.PROVOST@VESTIAIRECOLLECTIVE.COM,Small,...,0.0,0.0,9.536740e-07,0.0,1.0,1,0,0,0,SUCCESS
2,2,01a815b0-3201-fbc4-0000-27695f9a0042,SELECT * FROM DWH_INT.FCT_VC_INT_BS_MESSAGE_NE...,[],"[\n ""DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW""\n]",[],SELECT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Small,...,0.0,0.0,0.000000e+00,0.0,300.0,0,0,0,0,SUCCESS
3,3,01a81605-3201-fbc5-0000-27695f9c632e,SELECT TO_DATE(DATE_PUSH_SENT) AS DATE...,"[\n ""JOIN""\n]","[\n ""DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATI...","[\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,NaN,...,0.0,0.0,0.000000e+00,0.0,NaN,0,0,0,0,SUCCESS
4,4,01a815ec-3201-fbf1-0000-27695f9b7f06,insert into PRODUCTION.DWH_CRM.FCT_VC_CRM_BRAZ...,[],"[\n ""DWH_CRM.FCT_VC_CRM_BRAZE_API_UPDATE_CUST...",[],INSERT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Medium,...,0.0,0.0,1.293025e-01,0.0,3045233.0,3037692,0,0,0,SUCCESS


In [77]:
# for col in df.columns:
#     print(col)

df['warehouse_size'].unique()

array(['Small', nan, 'Medium', 'X-Small'], dtype=object)

In [17]:
df_tables = df[['query_id', 'tables_scan_list']].copy()
df_tables.dtypes
df_tables['tables_scan_list'] = df_tables['tables_scan_list'].apply(lambda x: eval(x.strip()))
df_tables = df_tables.rename(columns={'tables_scan_list': 'table_name'})
df_tables = df_tables.explode('table_name')
df_tables.head()

,query_id,table_name
0,01a815ab-3201-fbbf-0000-27695f99ba56,DWH_OPS.#Tableau_101782_1_Connect_CheckCreateT...
1,01a8165b-3201-fbc5-0000-27695f9ecb9a,DWH_PRD.#Tableau_3_1_Connect_CheckCreateTempTa...
2,01a815b0-3201-fbc4-0000-27695f9a0042,DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW
3,01a81605-3201-fbc5-0000-27695f9c632e,DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATION
3,01a81605-3201-fbc5-0000-27695f9c632e,DWH_MKT.DIM_VC_MKT_CRM_CAMPAIGNS_CATEGORY


In [19]:
storage_query = get_data_from_snow(
"""
select SCHEMA_NAME || '.' || TABLE_NAME as table_name
     , TB_ACTIVE * 1000                 as storage_gb
from meta.OPTIMIZATION.TABLE_STORAGE
where DATABASE_NAME = 'PRODUCTION'
"""
)
df_storage.head()

,table_name,storage_gb
0,DWH_MKT.FCT_VC_MKT_ATTRIBUTION_PRE_RESULTS,0.337
1,DWH_INT.FCT_VC_INT_MMAO_RLT,1.593
2,STAGING.FCT_VC_OPS_STAGING_CURATION_EVENTS_RLT,55.345
3,STAGING.DIM_VC_STAGING_PRODUCT_RANK,1.358
4,STAGING.DIM_VC_PRD_PRODUCT_CURATION_DATETIMES,2.959


In [27]:
if os.path.isfile(table_file):
    df_storage = pd.read_csv(table_file, lineterminator='\n', header=None)
else:
    df_storage = get_data_from_snow(sql)
#     df = df.replace(r'\n',' ', regex=True)
    df_storage.query_text = df.query_text.replace(r'\n|"',' ', regex=True)
    df_storage.tables_scan_list = df.tables_scan_list.replace(r'\n',' ', regex=True)
#     to_replace=r'^ba.$'
    df_storage.to_csv(fp + '/fct_query_history_lin_reg.csv', quoting=csv.QUOTE_MINIMAL)
    
df_storage.rename(columns={0: 'table_name'}, inplace=True)
df_storage.rename(columns={1: 'storage_gb'}, inplace=True)

df_storage.head()

,table_name,storage_gb
0,STAGING.TMP__FCT_VC_COUNTERFEIT_FEATURES_RT__C...,1.512
1,DWH_PRD.S_MARIADB_PRODUIT_LAST10DAYS_RLT,149.053
2,DWH_FIN.FCT_VC_FIN_HYPERWALLET,5.357
3,DWH_OPS.FCT_VC_OPS_TICKET_SATISFACTION,0.640
4,DWH_PRD.DIM_VC_PRD_PRODUCT_MASTERFEED,34.774


In [32]:
len(df_storage.index)
df_storage.dtypes

table_name     object
storage_gb    float64
dtype: object

In [28]:
df_merge = df_tables.merge(df_storage)
df_agg = df_merge.groupby(by=['query_id'], as_index=False).sum()
df_agg.rename(columns={'storage_gb': 'total_scan_potential_gb'}, inplace=True)
df_deduped = df_merge.drop_duplicates().groupby(by=['query_id'], as_index=False).sum()
df_deduped.rename(columns={'storage_gb': 'distinct_scan_gb'}, inplace=True)
df_scan_info = df_deduped.merge(df_agg)
# print(df_deduped.head())
df_scan_info.head()

C:\Users\phill\AppData\Local\Temp\ipykernel_9820\2529886402.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_agg = df_merge.groupby(by=['query_id'], as_index=False).sum()
C:\Users\phill\AppData\Local\Temp\ipykernel_9820\2529886402.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_deduped = df_merge.drop_duplicates().groupby(by=['query_id'], as_index=False).sum()


,query_id,distinct_scan_gb,total_scan_potential_gb
0,01a801a4-3201-f840-0000-27695f11fb02,74.944,74.944
1,01a801a4-3201-f840-0000-27695f11fc5a,75.010,75.010
2,01a801a4-3201-f883-0000-27695f12049a,75.010,75.010
3,01a801a4-3201-f93d-0000-27695f121226,30.647,30.647
4,01a801a4-3201-f93d-0000-27695f12133a,75.010,75.010


In [35]:
df = df.merge(df_scan_info)
df['nb_joins'] = df.join_list.apply(lambda x: len(literal_eval(x)))
df['nb_aggregations'] = df.aggregate_list.apply(lambda x: len(literal_eval(x)))
df.head()

,Unnamed: 0,query_id,query_text,join_list,tables_scan_list,aggregate_list,query_type,query_load_percent,user_name,warehouse_size,...,rows_produced,rows_inserted,rows_deleted,rows_updated,rows_unloaded,execution_status,distinct_scan_gb,total_scan_potential_gb,nb_joins,nb_aggregations
0,2,01a815b0-3201-fbc4-0000-27695f9a0042,SELECT * FROM DWH_INT.FCT_VC_INT_BS_MESSAGE_NE...,[],"[\n ""DWH_INT.FCT_VC_INT_BS_MESSAGE_NEW""\n]",[],SELECT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Small,...,300.0,0,0,0,0,SUCCESS,22.429,22.429,0,0
1,3,01a81605-3201-fbc5-0000-27695f9c632e,SELECT TO_DATE(DATE_PUSH_SENT) AS DATE...,"[\n ""JOIN""\n]","[\n ""DWH_CRM.SUM_VC_CRM_BRAZE_PUSH_NOTIFICATI...","[\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,NaN,...,NaN,0,0,0,0,SUCCESS,1.878,1.878,1,1
2,4,01a815ec-3201-fbf1-0000-27695f9b7f06,insert into PRODUCTION.DWH_CRM.FCT_VC_CRM_BRAZ...,[],"[\n ""DWH_CRM.FCT_VC_CRM_BRAZE_API_UPDATE_CUST...",[],INSERT,100.0,CLARA.MICHEL@VESTIAIRECOLLECTIVE.COM,Medium,...,3045233.0,3037692,0,0,0,SUCCESS,113.514,113.514,0,0
3,5,01a815bc-3201-fbc4-0000-27695f9a0d8e,SELECT DATE_SESSION date --- ALREADY D...,"[\n ""JOIN""\n]","[\n ""DWH_TRK.FCT_VC_TRK_DAU_TRAFFICSOURCE"",\n...","[\n ""DISTINCT"",\n ""GROUP BY""\n]",SELECT,NaN,ZHIXUAN.FANG@VESTIAIRECOLLECTIVE.COM,NaN,...,NaN,0,0,0,0,SUCCESS,61.103,61.103,1,2
4,7,01a81583-3201-fbc4-0000-27695f981b6a,"SELECT DATE_AGGREGATION AS DATE_EVENT, ...","[\n ""JOIN"",\n ""JOIN""\n]","[\n ""DWH_MKT.SUM_VC_MKT_ATTRIBUTION_PERFORMAN...","[\n ""group by""\n]",SELECT,NaN,NITHESH.RAMANNA@VESTIAIRECOLLECTIVE.COM,NaN,...,NaN,0,0,0,0,SUCCESS,2.631,2.631,2,1


In [55]:
# for col in df.columns:
#     print(col)
x=df[['total_scan_potential_gb', 'nb_joins', 'nb_aggregations']].to_numpy()
# x=df[['total_scan_potential_gb']].to_numpy()
y=df[['execution_time']]
print(x,y)

[[ 22.429]
 [  1.878]
 [113.514]
 ...
 [ 18.958]
 [ 32.037]
 [  1.212]]        execution_time
0               0.624
1               0.002
2              17.326
3               0.001
4               0.001
...               ...
10990           0.001
10991           0.001
10992           0.001
10993           0.213
10994           0.002

[10995 rows x 1 columns]


In [56]:
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)
model = LinearRegression()
reg_model = model.fit(x_train,y_train)

In [57]:
r_sq = model.score(x_train, y_train)
print(r_sq)

0.029997167944631475
